In [17]:
import os

SEASONS = list(range(2016,2023))
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [18]:
SEASONS

[2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [32]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [33]:
async def get_html(url, selector, sleep=5, retries=3):
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [35]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [9]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Refere

In [30]:
standings_files = os.listdir(STANDINGS_DIR)

In [ ]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
import pandas as pd

for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]
    
    for f in files:
        filepath = os.path.join(STANDINGS_DIR, f)
        
        await scrape_game(filepath)

Wizards vs Timberwolves, March 13, 2017 | Basketball-Reference.com
Hawks vs Spurs, March 13, 2017 | Basketball-Reference.com
Clippers vs Jazz, March 13, 2017 | Basketball-Reference.com
Lakers vs Nuggets, March 13, 2017 | Basketball-Reference.com
Magic vs Kings, March 13, 2017 | Basketball-Reference.com
Pistons vs Cavaliers, March 14, 2017 | Basketball-Reference.com
Thunder vs Nets, March 14, 2017 | Basketball-Reference.com
Pacers vs Knicks, March 14, 2017 | Basketball-Reference.com
Trail Blazers vs Pelicans, March 14, 2017 | Basketball-Reference.com
76ers vs Warriors, March 14, 2017 | Basketball-Reference.com
Hornets vs Pacers, March 15, 2017 | Basketball-Reference.com
Mavericks vs Wizards, March 15, 2017 | Basketball-Reference.com
Timberwolves vs Celtics, March 15, 2017 | Basketball-Reference.com
Jazz vs Pistons, March 15, 2017 | Basketball-Reference.com
Pelicans vs Heat, March 15, 2017 | Basketball-Reference.com
Grizzlies vs Bulls, March 15, 2017 | Basketball-Reference.com
Lakers vs 

In [ ]:
import pandas as pd

stats = pd.read_html(html)

In [ ]:
players = stats[0]

In [ ]:
summary = players[["Player", "Age", "Tm"]].groupby("Player", group_keys=False).agg(['count', 'min'])

In [ ]:
summary.columns = ["team_count", "age", "tc", "team"]